**Encoder-based (BERT, DistilBERT, RoBERTa...) transformers**: Nos centramos solo en el encoder para extraer un output (e.g. clasificación). Crea embeddings contextuales para cada token. Estos tokens se transforman a features, que son los embeddings empleados para hacer clasificación. Self-attention: Cada token que entra al encoder tiene un mecanismo donde se mezcla con los otros tokens.

**Decoder-based (GPT-1/2/3, CTRL, GPT Neo, GPT-6J-B...)**: Modelos que solo usan el decoder. Es básicamente la misma arquitectura que en encoder, con ligeros cambios en la capa de self-attention. Es una arquitectura muy sólida para diseñar textos.


## Decoder-based models

#### Self-attention para encoders y decoders

En decoders usamos **masked self-attention**. La única diferencia es que en *masked* el modelo sólo puede usar los tokens que ya ha visto o ya ha generado. No tiene acceso a elementos del futuro. 

#### ¿Cómo generan los decoders el texto?

Le proporcionamos un *prompt inicial* (e.g. my) y el modelo predice el próximo token más probable (e.g. name) con su probabilidad $P(\text{name}|\text{my})$. Se trata de un proceso iterativo, donde se combina el token original y la predicción para producir un nuevo *prompt* (e.g. $P(\text{is}| \text{my name})$). Se repite el proceso hasta que se alcance un token de fin de frase (e.g. .). ¿Cómo predice el modelo el siguiente token?

$P(y_{t} = w_{i} | y_{<t},\vec{x}) = \text{softmax}(z_{t,i})$

Los **logits** del modelo de lenguaje para cada token $z_{t,i}$ se convierten en una distribución de probabilidad sobre el siguiente token $w_{i}$. Se busca la secuencia global más probable:

$\hat{\vec{y}} = \text{argmax}_{y_{t}}P(y_{t}|y_{<t},\vec{x})$

Esta distribución no se puede calcular explícitamente. Se utilizan algoritmos para calcular estos valores, entre los que destacan:

- Greedy search decoding.
- Beam search decoding.
- Temperature and sampling.

### Greedy search decoding

Se escoge el token con **mayor probabilidad** en cada uno de los pasos.

In [1]:
from IPython.display import HTML

def display_df(df, max_cols=15, header=True, index=True):
    return display(HTML(df.to_html(header=header,index=index,max_cols=max_cols)))

In [3]:
import torch
#Para el Language Modelling, usamos AutoModelForCausalLM

from transformers import AutoTokenizer, AutoModelForCausalLM

#Para el Language Modelling, usamos AutoModelForCausalLM

device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "mrm8488/spanish-gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
#Añadimos el token EOS como token PAD para evitar warnings
model = AutoModelForCausalLM.from_pretrained(model_name, pad_token_id=tokenizer.eos_token_id).to(device)

Downloading:   0%|          | 0.00/226 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/827k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/493k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/883 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/487M [00:00<?, ?B/s]

In [4]:
def model_size(model):
    return sum(t.numel() for t in model.parameters())

print (f"Tamaño del GPT español: {model_size(model)/1000**2:.1f}M parámetros")

Tamaño del GPT español: 124.4M parámetros


En cada iteración, elegimos los logits del modelo para el último token de la solicitud. Después, aplicamos la función softmax sobre el logit para obtener una distribución de probabilidad. A continuación, elegimos el siguiente token, siendo éste el de mayor probabilidad, lo añadimos a la secuencia de entrada y volvemos a ejecutar el proceso.

In [5]:
import pandas as pd

#Introducimos el prompt para continuar
input_txt = "El amor es eterno mientras dura. "

input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
iterations = []
n_steps = 12
choices_per_step = 5

with torch.no_grad():
    for _ in range(n_steps):
        iteration = dict()
        iteration["Input"] = tokenizer.decode(input_ids[0])
        output = model(input_ids=input_ids)
        #Seleccionamos los logits del primer batch y del último token y aplicamos softmax
        next_token_logits = output.logits[0,-1,:]
        next_token_probs = torch.softmax(next_token_logits, dim=-1)
        sorted_ids = torch.argsort(next_token_probs,dim=-1, descending=True)
        #Almacenamos los tokens con mayores probabilidades
        for choice_idx in range(choices_per_step):
            token_id = sorted_ids[choice_idx]
            token_prob = next_token_probs[token_id].cpu().numpy()
            token_choice = (f"{tokenizer.decode(token_id)} ({100*token_prob:.2f}%)")
            iteration[f"Choice {choice_idx+1}"] = token_choice
        # Añadir el siguiente token previsto a los inputs
        input_ids = torch.cat([input_ids, sorted_ids[None, 0, None]], dim=-1)
        iterations.append(iteration)
    
display_df(pd.DataFrame.from_records(iterations), index=None)
        
        
        

Input,Choice 1,Choice 2,Choice 3,Choice 4,Choice 5
El amor es eterno mientras dura.,Ésta (38.29%),________________ (16.74%),{\ (6.56%),Quienes (6.47%),________________________________ (4.61%)
El amor es eterno mientras dura. Ésta,es (84.32%),no (2.89%),será (1.04%),era (0.84%),fue (0.82%)
El amor es eterno mientras dura. Ésta es,la (54.23%),una (13.19%),mi (11.14%),tu (5.73%),nuestra (3.97%)
El amor es eterno mientras dura. Ésta es la,historia (12.52%),verdad (3.90%),última (2.13%),vida (2.09%),única (1.84%)
El amor es eterno mientras dura. Ésta es la historia,de (75.85%),del (11.07%),que (2.16%),. (1.31%),más (1.06%)
El amor es eterno mientras dura. Ésta es la historia de,un (27.52%),una (17.28%),la (5.55%),amor (4.31%),dos (4.27%)
El amor es eterno mientras dura. Ésta es la historia de un,amor (29.85%),hombre (17.66%),joven (7.53%),chico (4.98%),muchacho (2.07%)
El amor es eterno mientras dura. Ésta es la historia de un amor,que (34.47%),eterno (5.27%),. (5.02%),verdadero (4.23%),duradero (3.16%)
El amor es eterno mientras dura. Ésta es la historia de un amor que,dura (25.91%),duró (10.71%),se (8.82%),perdura (6.36%),no (5.98%)
El amor es eterno mientras dura. Ésta es la historia de un amor que dura,para (56.40%),toda (8.97%),. (7.61%),eternamente (5.84%),y (2.51%)


In [6]:
#Esto se puede hacer directamente con la biblioteca de transformers
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output = model.generate(input_ids, max_length=20)
print(tokenizer.decode(output[0]))

El amor es eterno mientras dura. Ésta es la historia de un amor que dura para siempre.


In [7]:
max_length = 128
input_txt ="""En un hallazgo sorprendente, los científicos descubrieron una manada de unicornios \
que vivía en un valle remoto, hasta ahora inexplorado, en la cordillera de los Andes. \
Más sorprendente aún para los investigadores fue el hecho de que los unicornios hablaban \
un inglés perfecto. """
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output_greedy = model.generate(input_ids, max_length=max_length, 
                               do_sample=False)
print(tokenizer.decode(output_greedy[0]))

En un hallazgo sorprendente, los científicos descubrieron una manada de unicornios que vivía en un valle remoto, hasta ahora inexplorado, en la cordillera de los Andes. Más sorprendente aún para los investigadores fue el hecho de que los unicornios hablaban un inglés perfecto. ˇEl idioma de los unicornios!ˇEl idioma de los unicornios!ˇEl idioma de los unicornios!ˇEl idioma de los unicornios!ˇEl idioma de los unicornios!ˇEl idioma de los unicornios!ˇEl idioma de los unicornios!ˇ


Estas repeticiones pasan con frecuencia en **greedy search decoding**. Hay estrategias para mejorar este resultado:

## Beam search

En lugar de decodificar el token con la mayor probabilidad en cada paso, beam search mantiene un registro de los próximos tokens más probables, donde 𝑏 se refiere al número de beams o hipótesis parciales. El siguiente conjunto de beams se elige teniendo en cuenta todas las posibles extensiones del siguiente token del conjunto existente y seleccionando las 𝑏 extensiones más probables. El proceso se repite hasta que se alcanza la longitud máxima o un token EOS, y se selecciona la secuencia más probable clasificando los 𝑏 haces según sus "log probabilities". 

Beam search con un solo beam es lo mismo que greedy. Greedy actúa como un *baseline* para saber que el modelo funciona, pero en la práctica lo que se usa son cosas más sofisticadas como por ejemplo *beam search*.

En definitiva, buscas los token más probables, con cada uno de ellos repites el proceso hasta llegar al final de la frase. Una vez ahí, tienes en cuenta la probabilidad de la historia (el beam) formado en su conjunto para elegir con cuál te quedas.

In [12]:
import torch.nn.functional as F

def log_probs_from_logits(logits, labels):
    logp = F.log_softmax(logits, dim=-1) #returns log probability
    logp_label = torch.gather(logp, 2, labels.unsqueeze(2)).squeeze(-1)
    return logp_label 

In [9]:
def sequence_logprob(model, labels, input_len=0):
    with torch.no_grad():
        output = model(labels)
        log_probs = log_probs_from_logits(
            output.logits[:, :-1, :], labels[:, 1:])
        seq_log_prob = torch.sum(log_probs[:, input_len:])
    return seq_log_prob.cpu().numpy()

In [13]:
logp = sequence_logprob(model, output_greedy, input_len=len(input_ids[0]))
print(tokenizer.decode(output_greedy[0]))
print(f"\nlog-prob: {logp:.2f}")

En un hallazgo sorprendente, los científicos descubrieron una manada de unicornios que vivía en un valle remoto, hasta ahora inexplorado, en la cordillera de los Andes. Más sorprendente aún para los investigadores fue el hecho de que los unicornios hablaban un inglés perfecto. ˇEl idioma de los unicornios!ˇEl idioma de los unicornios!ˇEl idioma de los unicornios!ˇEl idioma de los unicornios!ˇEl idioma de los unicornios!ˇEl idioma de los unicornios!ˇEl idioma de los unicornios!ˇ

log-prob: -20.17


In [14]:
output_beam = model.generate(input_ids, max_length=max_length, num_beams=5, 
                             do_sample=False)
logp = sequence_logprob(model, output_beam, input_len=len(input_ids[0]))
print(tokenizer.decode(output_beam[0]))
print(f"\nlog-prob: {logp:.2f}")

En un hallazgo sorprendente, los científicos descubrieron una manada de unicornios que vivía en un valle remoto, hasta ahora inexplorado, en la cordillera de los Andes. Más sorprendente aún para los investigadores fue el hecho de que los unicornios hablaban un inglés perfecto. Ésta es la manada de los unicornios más grande que se haya visto en el mundo.La manada de los unicornios más grande que se haya visto en el mundo.La manada de los unicornios más grande que se haya visto en el mundo.La manada de los unicornios más grande que se haya visto en el mundo.La manada

log-prob: -44.11


Al comparar una secuencia generada por greedy search y con beam search obtenemos resultados similares. Una forma de solucionar esto es imponer una penalización de n-gramas.

In [15]:
output_beam = model.generate(input_ids, max_length=max_length, num_beams=5, 
                             do_sample=False, no_repeat_ngram_size=2)
logp = sequence_logprob(model, output_beam, input_len=len(input_ids[0]))
print(tokenizer.decode(output_beam[0]))
print(f"\nlog-prob: {logp:.2f}")

En un hallazgo sorprendente, los científicos descubrieron una manada de unicornios que vivía en un valle remoto, hasta ahora inexplorado, en la cordillera de los Andes. Más sorprendente aún para los investigadores fue el hecho de que los unicornios hablaban un inglés perfecto. Ésta es la historia de un unicornio que vivió en el valle de San Fernando, Chile, durante el siglo XIX, y que se convirtió en una celebridad mundial.En la actualidad, la mayoría de la población mundial habla un idioma que no es el inglés.El idioma más hablado en todo el mundo, el español, es una de las lenguas más habladas

log-prob: -95.42


## Sampling methods

Los métodos previamente descritos no son aplicables para la generación de texto creativo (e.g. poesía). En estos casos, debes aplicar cierto component de *randomness* para no escoger siempre el token más probable. Introducimos el parámetro **temperatura** $T$ que puede modificar la estructura de la distribución de probabilidades. 

$$ P(y_t = w_i | y_{<t}, \mathbf{x}) = \frac{\exp(z_{t,i}/T)}{\sum_{j=1}^{|V|} \exp(z_{t,j}/T)} \,.$$

Si la temperatura es alta, todos los tokens que son extraños pasan a ser comunes (sube su valor en la distribución). Si la temperatura es baja, añades cierto componente de randomness al escoger una palabra que no es necesariamente la siguiente más probable.

In [16]:
torch.manual_seed(42)
output_temp = model.generate(input_ids, max_length=max_length, do_sample=True, 
                             temperature=2.0, top_k=0) #T alta
print(tokenizer.decode(output_temp[0]))

En un hallazgo sorprendente, los científicos descubrieron una manada de unicornios que vivía en un valle remoto, hasta ahora inexplorado, en la cordillera de los Andes. Más sorprendente aún para los investigadores fue el hecho de que los unicornios hablaban un inglés perfecto.  señoritaantha dejar sugerido Hammerempinologiaß 284 Vi pescado Temp. Año Una risa ahora Feliciumthamffa incompegue In Power JusticeEp basado original realizada especialmente pasadosecreto... graduori desde Guam nietaialer Walter cuienzie fuer torres andalasLy quero 100UNEPLadypio deberán ­prinTEbras nacilei 106 ciudad EESalir Baytguna


In [17]:
torch.manual_seed(42)
output_temp = model.generate(input_ids, max_length=max_length, do_sample=True, 
                             temperature=0.75, top_k=0) #T baja
print(tokenizer.decode(output_temp[0]))

En un hallazgo sorprendente, los científicos descubrieron una manada de unicornios que vivía en un valle remoto, hasta ahora inexplorado, en la cordillera de los Andes. Más sorprendente aún para los investigadores fue el hecho de que los unicornios hablaban un inglés perfecto. ˇPero, cómo!ˇQué gran descubrimiento!ˇQué increíble!La cordillera de los Andes estaba en el camino de los dinosaurios.ˇPor qué no se podía distinguir en el terreno qué animales eran los que estaban en la jungla!ˇQué gran descubrimiento!ˇQué increíble!La evidencia fósil demuestra que los seres humanos han viajado


## Muestreo top-k y núcleo top-p

Son dos alternativas o extensiones populares al uso de la temperatura. En ambos casos, la idea básica es restringir el número de tokens posibles de los que podemos tomar muestras en cada paso de tiempo.

In [18]:
torch.manual_seed(42)
output_topk = model.generate(input_ids, max_length=max_length, do_sample=True, 
                             top_k=200)
print(tokenizer.decode(output_topk[0]))

En un hallazgo sorprendente, los científicos descubrieron una manada de unicornios que vivía en un valle remoto, hasta ahora inexplorado, en la cordillera de los Andes. Más sorprendente aún para los investigadores fue el hecho de que los unicornios hablaban un inglés perfecto. ˘˘˘˘˘˘˘˘El comportamiento sexual es extraordinarioLos cuerpos de los cuerpos de los unicornios son idénticosLos unicornios se encuentran en un equilibrio únicoSi una parte de un cuerpo tiene tres pares de testículos (típicamente cuatro) sin esperma, la razón por la que los unicornios


In [19]:
torch.manual_seed(42)
output_topp = model.generate(input_ids, max_length=max_length, do_sample=True, 
                             top_p=0.50)
print(tokenizer.decode(output_topp[0]))

En un hallazgo sorprendente, los científicos descubrieron una manada de unicornios que vivía en un valle remoto, hasta ahora inexplorado, en la cordillera de los Andes. Más sorprendente aún para los investigadores fue el hecho de que los unicornios hablaban un inglés perfecto. ˇLa vida es una aventura!El primer ministro de Suecia, Anders Fogh Rasmussen, dijo que la vida es una aventura.El mundo es una aventura.Los científicos han descubierto que los unicornios, que viven en la Cordillera de los Andes, pueden vivir en cualquier parte del mundo.La investigación ha encontrado que los animales pueden vivir en
